In [2]:
import re

# Parent classes

In [237]:
def is_available_move(c, color: str, coords: tuple) -> bool:   
    """
    Check if the given move is available.

    coords: tuple of coordinates in format (y, x), where y - number of row, x - number of column
    """
    y = coords[0]
    x = coords[1]
    if all([coord in range(8) for coord in coords]): # checking for going beyond the boundaries of the board
        chessman_on_cell = c.field[y][x]
        
        if chessman_on_cell == "." or chessman_on_cell.color != color:
                # If the cell is empty or there stands enemy - available move
            return True
    
    return False

In [238]:
class Chessman:
    """ 
    Class describes parent class for each chess pieces.

    x: horizontal coord. on the chessboard;
    y: vertical coord. on the chessboard;
    color: color of the chessman ("black" or "white");
    symb: symbol, which is used to visualize chessman on the chessboard;
    
    make_move: describes which moves chessman can make on the chessboard; description of how the figure walks

    """
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = None
        self.selected = False
        self.moves_map = []

    def __str__(self):
        return self.symb if self.color == "black" else self.symb.upper()

    def set_color(self, color: str) -> None:
        self.color = color

    def set_y(self, y: str) -> None:
        self.y = y

    def make_move(self) -> None:
        pass

    def move_description(self) -> None:
        pass

    
    def fill_moves_map(self, c, coords: tuple, on_own_part=None) -> None:
        y = coords[0]
        x = coords[1]

        if self.__class__.__name__ == "Pawn":
            curr_coords = coords # coords for tracking the possible move of the chessman

            if is_available_move(c=c, color=self.color, coords=self.make_move(curr_coords)):
                curr_coords = self.make_move(curr_coords)   
                self.moves_map.append(curr_coords)
                if on_own_part:
                    self.fill_moves_map(c=c, coords=curr_coords, on_own_part=False)

## Chessman's child classes

In [239]:
class Pawn(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'p'
        self.moves_map = [] # list contains all coords in tuple format (y, x), with cells where the chessman can move

    def make_move(self, coords: tuple) -> tuple: # coords in tuple format (y, x)
        direction = 1 if self.color == "white" else -1 # white chessmen on the board go down, black - up
        y = coords[0] + direction
        x = coords[1]
        return (y, x)

    def move_description(self, c: Chessboard):
        """ 
        One cell towards, two if the chessman on its half of board. 
        """
        
        on_own_part = self.x in range(4) if self.color == 'white' else self.x in range(5, 8) # check if the pawn on his half of board

        self.fill_moves_map(c=c, coords=(self.y, self.x), on_own_part=on_own_part)

In [240]:
class Rook(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'r'

In [241]:
class Knight(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'n'

In [242]:
class Bishop(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'b'

In [243]:
class King(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'k'

In [244]:
class Queen(Chessman):
    def __init__(self, x=None, y=None, color=None) -> None:
        self.x = x
        self.y = y
        self.color = color
        self.symb = 'q'

# Chessboard class

In [245]:
class Chessboard:
    """
    Class describes properties of the chessboard and its connection with chess pieces.

    field: first dimension is rows, second - columns. Contains blank squares (.) and pieces, each chessman in this list is an instance of the corresponding class;

    """

    def __init__(self) -> None:
        self.field = [["." for _ in range(8)] for _ in range(8)]
        self.letters2idx = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7} # dict for converting letters on the board to list's columns indexes
        self.nums2idx = dict(zip([str(8-i) for i in range(8)], [i for i in range(8)])) # dict for converting nums on the board to list's rows indexes
        self.current_color = 'white' # color whose turn it is to make a move

        def fill_field() -> None:
            """
            Fill the chessboard with the pieces of both colors.
            """

            for color in ["black", "white"]:
                sp_line_idx, pawns_line_idx = (0, 1) if color == "white" else (7, 6) # Define the indexes of pawns' line and special line according to the color

                pawns = [Pawn(x=x, y=pawns_line_idx, color=color) for x in range(8)]

                sec_line = [Rook(x=0), Knight(x=1), Bishop(x=2), Queen(x=3), King(x=4), Bishop(x=5), Knight(x=6), Rook(x=7)]

                for f in sec_line: # set color and y-coord for all 'special' chessmen
                    f.set_color(color)
                    f.set_y(sp_line_idx)


                self.field[sp_line_idx], self.field[pawns_line_idx] = sec_line, pawns
        
        fill_field()

    def show_desk(self) -> None:
        """ 
        Show the current state of the chessboard. 
        """
        print("   A B C D E F G H")
        print()
        for i in range(len(self.field)):
            print(f"{8-i}", end="  ")
            for j in range(len(self.field[i])):
                print(self.field[i][j], end=" ")
            print("", end=" ")
            print(f"{8-i}")
        print()
        print("   A B C D E F G H",)

    def select_the_chessman(self):
        """ 
        Describes chessman selection mechanism. 
        """
        try:
            cell = input("Enter the coordinates of the cell separated by a space in format 'Letter Number': ")
            if cell == "exit": return # exit from the game
 
            prog = re.compile("[1-8A-Ha-h]")
            cells = prog.findall(cell)

            assert len(cells) == 2 and (cells[0].isdigit() and cells[1].isalpha() or # check if the entered data is valid
                                        cells[1].isdigit() and cells[0].isalpha())

            b_n, b_l = (cells[0], cells[1]) if cells[0].isdigit() else (cells[1], cells[0]) # number, letter of the cell

            row_idx, col_idx = self.nums2idx[b_n], self.letters2idx[b_l.upper()] # converting number and letter to corresponding indexes

            sel_ch = self.field[row_idx][col_idx] # get the chessman from field

            assert sel_ch != '.' and sel_ch.color == self.current_color # check if empty cell or chessman of opponent was selected

        except AssertionError:
            print("Incorrect data entered.")
            return self.select_the_chessman()

        sel_ch.move_description(self)
        return sel_ch.moves_map

In [246]:
d = Chessboard()
d.show_desk()
d.select_the_chessman()

   A B C D E F G H

8  R N B Q K B N R  8
7  P P P P P P P P  7
6  . . . . . . . .  6
5  . . . . . . . .  5
4  . . . . . . . .  4
3  . . . . . . . .  3
2  p p p p p p p p  2
1  r n b q k b n r  1

   A B C D E F G H
Enter the coordinates of the cell separated by a space in format 'Letter Number': A7


[(2, 0), (3, 0)]